# Importer les bibliothèques

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk

# Télécharger les stopwords français

In [ ]:
nltk.download("stopwords")
from nltk.corpus import stopwords
french_stopwords = stopwords.words("french")

# Charger et combiner les données des deux fichiers CSV

In [ ]:
# Charger les données 
fightclub_df = pd.read_csv("fightclub_critiques.csv")
interstellar_df = pd.read_csv("interstellar_critiques.csv")

# Ajouter une colonne pour identifier le film
fightclub_df["movie_title"] = "Fight Club"
interstellar_df["movie_title"] = "Interstellar"

# Combiner les deux films dans un seul DataFrame
df = pd.concat([fightclub_df, interstellar_df], ignore_index=True)
df

# prétraitement des données textuelles

In [ ]:
df.isnull().sum()

In [ ]:
# Supprimer les lignes où review_content est NaN
df = df.dropna(subset=["review_content"]).reset_index(drop=True)
df

### Nettoyage du texte

In [ ]:
def clean_text(text):
    text = re.sub(r'<[^>]+>', '', text)  # retire les balises HTML
    text = text.lower()
    return text

# Appliquer la fonction de nettoyage clean_text sur toutes les critiques
# et stocker le texte nettoyé dans une nouvelle colonne 'clean_review'

df["clean_review"] = df["review_content"].apply(clean_text)
df

### Calculer TF-IDF

In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words=french_stopwords)
tfidf_matrix = tfidf_vectorizer.fit_transform(df["clean_review"])

print("TF-IDF calculé pour", tfidf_matrix.shape[0], "critiques et", tfidf_matrix.shape[1], "mots/features.")

# Fonction : récupérer les critiques les plus similaires à une critique donnée


In [ ]:
def get_similar_reviews(critique_id, top_n=5, use_rating=False, rating_weight=0.5):
    """
    Récupère les top N critiques les plus similaires à une critique donnée,
    avec possibilité de prendre en compte le rating et le nombre de likes.

    Paramètres :
    - critique_id : ID de la critique de référence
    - top_n : nombre de critiques à retourner
    - use_rating : si True, pondère la similarité selon la note
    - rating_weight : poids de la note dans le score combiné (0 à 1)
    """
    # Vérifier que l'ID existe
    if critique_id not in df["id"].values:
        return f"Critique {critique_id} introuvable"
    
    # Récupérer l'indice de la critique
    idx = df.index[df["id"] == critique_id][0]
    movie = df.loc[idx, "movie_title"]
    rating_ref = df.loc[idx, "rating"]
    
    # Filtrer les critiques du même film
    movie_indices = df[df["movie_title"] == movie].index
    movie_tfidf = tfidf_matrix[movie_indices]
    
    # Calculer la similarité cosine
    sim_scores = cosine_similarity(tfidf_matrix[idx], movie_tfidf).flatten()
    
    # Exclure la critique elle-même
    sim_scores[df.index.get_indexer([idx])] = 0
    
    # Pondération par rating
    if use_rating:
        rating_diff = np.abs(df.loc[movie_indices, "rating"] - rating_ref)
        rating_score = 1 - (rating_diff / df["rating"].max())  # Normaliser entre 0 et 1
        sim_scores = (1 - rating_weight) * sim_scores + rating_weight * rating_score
    
    # Trier indices par score combiné
    top_indices = sim_scores.argsort()[::-1][:top_n]
    
    # Récupérer la critique d’entrée
    current_review = df.loc[[idx], ["id", "review_title", "review_content", "rating", "gen_review_like_count"]].copy()
    current_review["similarity"] = 1.0  # Similarité maximale avec elle-même
    
    # Récupérer les critiques similaires
    similar_reviews = df.iloc[movie_indices[top_indices]][
        ["id", "review_title", "review_content", "rating", "gen_review_like_count"]
    ].copy()
    similar_reviews["similarity"] = sim_scores[top_indices]
    
    # Trier secondairement par nombre de likes
    similar_reviews = similar_reviews.sort_values(by=["similarity", "gen_review_like_count"], ascending=[False, False])
    
    # Concaténer : critique d'entrée + résultats
    result = pd.concat([current_review, similar_reviews], ignore_index=True)
    
    return result.reset_index(drop=True)


### Exemple à tester 

In [ ]:
# Critique de référence : 
critique_id = df[df["movie_title"] == "Fight Club"]["id"].iloc[2]

# Top 5 critiques similaires en prenant en compte le rating
top_similaires = get_similar_reviews(critique_id, top_n=5, use_rating=True, rating_weight=0.3)

top_similaires

In [ ]:
# Afficher le texte complet dans toutes les colonnes
pd.set_option('display.max_colwidth', None)

# Maintenant afficher le DataFrame
top_similaires
